# task 4: guessing the secret sequence
this notebook acts as the "hacker" trying to guess the password through the network connection.

### 1. hardware initialization
make sure `task4.bit` and `task4.hwh` are uploaded to the same directory as this notebook.

In [ ]:
from pynq import Overlay
import time

# load the hardware overlay
ol = Overlay("task4.bit")

# map the gpios based on the block design
hacker_gpio = ol.axi_gpio_0
match_gpio  = ol.axi_gpio_1

### 2. helper functions
these functions handle the hardware-level communication protocol.

In [ ]:
def send_hacker_bit(val):
    """
    sends a single bit to the pl.
    protocol: trigger high -> trigger low.
    """
    # trigger high (bit 1 = 1) | data bit (bit 0 = val)
    hacker_gpio.write(0, 0x2 | (val & 1)) 
    time.sleep(0.01) # short delay for hardware to catch the edge
    
    # trigger low (bit 1 = 0) | data bit (bit 0 = val)
    hacker_gpio.write(0, 0x0 | (val & 1))
    
    # read the current match count from pl
    return match_gpio.read()

def reset_game():
    print("please press btn1 (set button) on the fpga to reset the secret...")
    time.sleep(3)
    print("ready to hack!")

### 3. the brute force attack
this loop sends the bit stream and displays the correct count alongside the stream history.

In [ ]:
def brute_force_attack():
    # 8-bit target sequence (set this exact sequence on the physical switches first!)
    target = "01001001" 
    guessed_stream = ""
    
    print("attempting to hack sequence...")
    
    for bit in target:
        guessed_stream += bit
        current_match = send_hacker_bit(int(bit))
        
        # display the stream and correct bits 
        print(f"stream: {guessed_stream} -> match count: {current_match}")
        
        if current_match == 8:
            print("success: system unlocked! (green leds should be on)")
            return
            
    print("hack finished. did we unlock it?")

# run the test
brute_force_attack()